In [1]:
# $\def\*#1{\mathbf{#1}}$
import os
import sys
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt

#os.listdir("../../")
sys.path.append("../../")

%matplotlib inline

# Statistical Network Models



Let $V=\{\theta_{1},\cdots,\theta_{2},\}$ be a countably infinite set of nodes with $\theta_i \in \mathbb{R}_{+}$. We represent the directed multigraph of interest  using an atomic measure on $\mathbb{R}^2_{+}$

\begin{equation}
 D = \sum^{\infty}_{i=1}\sum^{\infty}_{j=1}n_{ij}\delta_{(\theta_i,\theta_j)}
\end{equation}

Where $n_{ij}$ counts the number of directed edges from node $\theta_i$ to node $\theta_j$.  Our generative approach for modeling $D$ associates with each node $\theta_i$ a sociability parameter $w_i > 0$ defined via the atomic random measure

\begin{equation}
 W = \sum^{\infty}_{i=1} w_{i}\delta_{\theta_i}
\end{equation}

Which we take to be distributed according to a homogeneous CRM, $W \sim CRM(\rho,\lambda)$. Given $W$, $D$ is simply generated from a Poisson process (PP) with intensity given by the product measure $\tilde{W} = W \times W$. The generalized gamma process (GGP) is a flexible two parameter CRM, with interpretable parameters and remarkable conjugagy properties. The L\'evy intensity of the GGP is given by:

\begin{equation}
 \rho(dw) = \frac{1}{\Gamma(1-\sigma)}w^{-1 - \sigma}\exp(-\tau w)dw
\end{equation}

Notice the difference with the paper of BFRY priors were the Generalized Gamma Process levy intensity is defined as:

\begin{equation}
 \rho(dw) = \frac{\theta}{\Gamma(1-\alpha)}w^{-1 - \alpha}\exp(-\tau w)dw
\end{equation}

where the two parameters verify 
\begin{equation}
 (\sigma,\tau) \in (-\infty,0]\times(0,+\infty) \text{ or }  (\sigma,\tau) \in  (0,1) \times [0,+\infty)
\end{equation}


In [4]:
#SYMPY REFERENCE https://minireference.com/static/tutorials/sympy_tutorial.pdf
#$\def\*#1{\mathbf{#1}}$

from sympy import *
from sympy.interactive import printing
import sympy
from sympy import Eq
import sympy as sym
from sympy.abc import theta, phi, alpha, tau
from __future__ import division
from scipy.stats import pareto
from random import randint
from sympy import Symbol
#from mpmath import *

printing.init_printing(use_latex='mathjax')


K = 3 #Number of communities
nk = symbols("n_0:"+str(K),integer=True) 
K = Symbol("K",integer=True)
pk = Symbol("p",real=True)
lamb = Symbol("\\lambda",function=True)
theta = Symbol("\\theta")
F = symbols("\Gamma_0:"+str(K),function=True)
G = Symbol("\Gamma",function=True)
alpha = Symbol("\\alpha",function=True)
beta = Symbol("\\beta")
Alphas =  symbols("\\alpha_0:"+str(K))
x, y , w, s, c, u, a, b,v  = symbols("x y w s c u a b v")
sig = Symbol("\sigma")
E = [Symbol("\Gamma_{\\alpha_"+str(i)+"}",function=True)(x) for i in range(10)]
k, n = symbols("k n", integer=True)
t = symbols("t",real=True)
P = symbols("P",function=True)
Z = Symbol("Z")
z = Symbol("z",function=True)
p = Symbol("p",function=True)
N = Symbol("N",integer=True)
q =  Symbol("p",function=True)
Expectation = Symbol("\\mathbb{E}",function=True)

In [5]:
edge_probability = (p(u,v)**z(u,v))*((1-p(u,v))**(1-z(u,v)))
graph_probability = Product(Product(edge_probability,(v,1,N)),(u,1,N))
#backToSubIndiceDouble(backToSubIndiceDouble((graph_probability.subs(K,3)).doit(),p,(1 ,3,1)),z,(1,3,1))
graph_probability

  N      N                                             
┬────┬ ┬────┬                                          
│    │ │    │               -z(u, v) + 1  z(u, v)      
│    │ │    │ (-p(u, v) + 1)            ⋅p       (u, v)
│    │ │    │                                          
u = 1  v = 1                                           

In [33]:
beta

\beta

$\def\*#1{\mathbf{#1}}$

Our first step will be a variational approximation for a simple undirected graph. We define the prior values of the sociabilities $w_{i}$ from a *finite-dimensional process* as defined from 

$$
\mu_{K}(\cdot) = \sum^{K}_{k=1} W_k \delta_{\theta_k}
$$

Where the probabilities of the $W_k$ are defined from  the BFRY priors:

$$
\text{BRS}(c,\tau,\sigma) = \frac{\sigma s^{- \sigma - 1} (- s e^{- (\frac{\sigma}{c})^{\frac{1}{\sigma}}} + 1) e^{- \tau \theta}}{(- \tau^{\sigma} + (\tau + (\frac{\sigma}{c})^{\frac{1}{\sigma}})^{\sigma}) \Gamma{ (- \sigma + 1  )}}
$$

For the variational approximation we need:

\begin{equation}
P(\*Z,\*W) = P(\*Z|\*W)P(\*W)
\end{equation}

In [34]:
sociabilities_probabilities = 1 - exp(-2*w(u)*w(v))
sociabilities_probabilities

     -2⋅w(u)⋅w(v)
1 - ℯ            

In [35]:
likehood_of_graph = graph_probability.subs(p(u,v),sociabilities_probabilities)

In [36]:
likehood_of_graph

   N        N                                                           
┬──────┬ ┬──────┬                                                       
│      │ │      │                    z(u, v)                -z(u, v) + 1
│      │ │      │ ⎛     -2⋅w(u)⋅w(v)⎞        ⎛ -2⋅w(u)⋅w(v)⎞            
│      │ │      │ ⎝1 - ℯ            ⎠       ⋅⎝ℯ            ⎠            
│      │ │      │                                                       
 u = 1    v = 1                                                         

In [37]:
prior_of_weights = Product(bfry_density_function(w(k))*lamb(theta(k)),(k,1,K))

We requiere at least one sociability per node, we can have K > N, and in consequence *unseen nodes*, values of $\theta_k$ which are part of the *finite-dimensional process* but absent from the graph, since the sociabilities failed to create an edge 

In [38]:
fantom_edge_probability = (1-p(u,v))
likehood_of_fantom_graph = Product(Product(fantom_edge_probability,(v,N+1,K)),(u,N+1,K))
likehood_of_fantom_graph = likehood_of_fantom_graph.subs(p(u,v),sociabilities_probabilities)
likehood_of_fantom_graph

    K         K                  
  ┬────┬    ┬────┬               
  │    │    │    │   -2⋅w(u)⋅w(v)
  │    │    │    │  ℯ            
  │    │    │    │               
u = N + 1 v = N + 1              

In [39]:
joint_probability_of_graph = likehood_of_graph*prior_of_weights*likehood_of_fantom_graph
joint_probability_of_graph

                                                ⎛   N        N                
⎛  K                                          ⎞ ⎜┬──────┬ ┬──────┬            
⎜┬───┬                                        ⎟ ⎜│      │ │      │            
⎜│   │ \lambda(\theta(k))⋅g_{1/K,\sigma}(w(k))⎟⋅⎜│      │ │      │ ⎛     -2⋅w(
⎜│   │                                        ⎟ ⎜│      │ │      │ ⎝1 - ℯ     
⎝k = 1                                        ⎠ ⎜│      │ │      │            
                                                ⎝ u = 1    v = 1              

                                           ⎞     K         K                  
                                           ⎟   ┬────┬    ┬────┬               
        z(u, v)                -z(u, v) + 1⎟   │    │    │    │   -2⋅w(u)⋅w(v)
u)⋅w(v)⎞        ⎛ -2⋅w(u)⋅w(v)⎞            ⎟⋅  │    │    │    │  ℯ            
       ⎠       ⋅⎝ℯ            ⎠            ⎟   │    │    │    │               
                                           ⎟ u = N 

In [40]:
bfry_density = bfry_density.subs(alpha,sig)
bfry_density = bfry_density.subs(c,1/K)
bfry_density_evaluated_wk = bfry_density.subs(s,w(k))

In [41]:
joint_probability_of_graph = joint_probability_of_graph.subs(lamb(theta(k)),1)

In [42]:
expand_log(log(joint_probability_of_graph),force=True)

                                     ⎛   N        N                           
   ⎛  K                       ⎞      ⎜┬──────┬ ┬──────┬                       
   ⎜┬───┬                     ⎟      ⎜│      │ │      │                    z(u
log⎜│   │ g_{1/K,\sigma}(w(k))⎟ + log⎜│      │ │      │ ⎛     -2⋅w(u)⋅w(v)⎞   
   ⎜│   │                     ⎟      ⎜│      │ │      │ ⎝1 - ℯ            ⎠   
   ⎝k = 1                     ⎠      ⎜│      │ │      │                       
                                     ⎝ u = 1    v = 1                         

                                ⎞      ⎛    K         K                  ⎞
                                ⎟      ⎜  ┬────┬    ┬────┬               ⎟
, v)                -z(u, v) + 1⎟      ⎜  │    │    │    │   -2⋅w(u)⋅w(v)⎟
     ⎛ -2⋅w(u)⋅w(v)⎞            ⎟ + log⎜  │    │    │    │  ℯ            ⎟
    ⋅⎝ℯ            ⎠            ⎟      ⎜  │    │    │    │               ⎟
                                ⎟      ⎝u = N + 1 v = N + 1            

# Algebra Log Graph

In [43]:
expandend_log_graph_likehood = expand_log(log(likehood_of_graph.subs(N,3).doit()),force=True).simplify()

A = backToSubIndiceDouble(expandend_log_graph_likehood,z,(1,4,1))
B = backToSubIndice(A,w,(1,4,1))
B

                                                                              
    2             2                                                           
2⋅w₁ ⋅z_1,1 - 2⋅w₁  + 2⋅w₁⋅w₂⋅z_1,2 + 2⋅w₁⋅w₂⋅z_2,1 - 4⋅w₁⋅w₂ + 2⋅w₁⋅w₃⋅z_1,3 

                                                                              
                                2             2                               
+ 2⋅w₁⋅w₃⋅z_3,1 - 4⋅w₁⋅w₃ + 2⋅w₂ ⋅z_2,2 - 2⋅w₂  + 2⋅w₂⋅w₃⋅z_2,3 + 2⋅w₂⋅w₃⋅z_3,

                                             ⎛          2⎞                    
                  2             2            ⎜     -2⋅w₁ ⎟            ⎛     -2
2 - 4⋅w₂⋅w₃ + 2⋅w₃ ⋅z_3,3 - 2⋅w₃  + z_1,1⋅log⎝1 - ℯ      ⎠ + z_1,2⋅log⎝1 - ℯ  

                                                                         ⎛    
⋅w₁⋅w₂⎞            ⎛     -2⋅w₁⋅w₃⎞            ⎛     -2⋅w₁⋅w₂⎞            ⎜    
      ⎠ + z_1,3⋅log⎝1 - ℯ        ⎠ + z_2,1⋅log⎝1 - ℯ        ⎠ + z_2,2⋅log⎝1 - 

      2⎞                                        

In [44]:
expandend_log_graph_likehood

                                                                              
   2                 2                                                        
2⋅w (1)⋅z(1, 1) - 2⋅w (1) + 2⋅w(1)⋅w(2)⋅z(1, 2) + 2⋅w(1)⋅w(2)⋅z(2, 1) - 4⋅w(1)

                                                                              
                                                                     2        
⋅w(2) + 2⋅w(1)⋅w(3)⋅z(1, 3) + 2⋅w(1)⋅w(3)⋅z(3, 1) - 4⋅w(1)⋅w(3) + 2⋅w (2)⋅z(2,

                                                                              
         2                                                                   2
 2) - 2⋅w (2) + 2⋅w(2)⋅w(3)⋅z(2, 3) + 2⋅w(2)⋅w(3)⋅z(3, 2) - 4⋅w(2)⋅w(3) + 2⋅w 

                                   ⎛         2   ⎞                            
                 2                 ⎜     -2⋅w (1)⎟              ⎛     -2⋅w(1)⋅
(3)⋅z(3, 3) - 2⋅w (3) + z(1, 1)⋅log⎝1 - ℯ        ⎠ + z(1, 2)⋅log⎝1 - ℯ        

                                                

In [45]:
take_w1 = collectTermsWithVariable(expandend_log_graph_likehood,w(1))
take_w1

                                                                              
   2                 2                                                        
2⋅w (1)⋅z(1, 1) - 2⋅w (1) + 2⋅w(1)⋅w(2)⋅z(1, 2) + 2⋅w(1)⋅w(2)⋅z(2, 1) - 4⋅w(1)

                                                                             ⎛
                                                                             ⎜
⋅w(2) + 2⋅w(1)⋅w(3)⋅z(1, 3) + 2⋅w(1)⋅w(3)⋅z(3, 1) - 4⋅w(1)⋅w(3) + z(1, 1)⋅log⎝

         2   ⎞                                                                
     -2⋅w (1)⎟              ⎛     -2⋅w(1)⋅w(2)⎞              ⎛     -2⋅w(1)⋅w(3
1 - ℯ        ⎠ + z(1, 2)⋅log⎝1 - ℯ            ⎠ + z(1, 3)⋅log⎝1 - ℯ           

                                                                    
)⎞              ⎛     -2⋅w(1)⋅w(2)⎞              ⎛     -2⋅w(1)⋅w(3)⎞
 ⎠ + z(2, 1)⋅log⎝1 - ℯ            ⎠ + z(3, 1)⋅log⎝1 - ℯ            ⎠

In [46]:
take_w1 = factor(take_w1,w(1))

In [47]:
take_w1

                                                                              
                 2                                                            
(2⋅z(1, 1) - 2)⋅w (1) + (2⋅w(2)⋅z(1, 2) + 2⋅w(2)⋅z(2, 1) - 4⋅w(2) + 2⋅w(3)⋅z(1

                                                  ⎛         2   ⎞             
                                                  ⎜     -2⋅w (1)⎟             
, 3) + 2⋅w(3)⋅z(3, 1) - 4⋅w(3))⋅w(1) + z(1, 1)⋅log⎝1 - ℯ        ⎠ + z(1, 2)⋅lo

                                                                              
 ⎛     -2⋅w(1)⋅w(2)⎞              ⎛     -2⋅w(1)⋅w(3)⎞              ⎛     -2⋅w(
g⎝1 - ℯ            ⎠ + z(1, 3)⋅log⎝1 - ℯ            ⎠ + z(2, 1)⋅log⎝1 - ℯ     

                                         
1)⋅w(2)⎞              ⎛     -2⋅w(1)⋅w(3)⎞
       ⎠ + z(3, 1)⋅log⎝1 - ℯ            ⎠

In [48]:
backToSubIndice(backToSubIndiceDouble(take_w1,z,(1,4,1)),w,(1,4,1))

                                                                              
  2                                                                           
w₁ ⋅(2⋅z_1,1 - 2) + w₁⋅(2⋅w₂⋅z_1,2 + 2⋅w₂⋅z_2,1 - 4⋅w₂ + 2⋅w₃⋅z_1,3 + 2⋅w₃⋅z_3

                      ⎛          2⎞                                           
                      ⎜     -2⋅w₁ ⎟            ⎛     -2⋅w₁⋅w₂⎞            ⎛   
,1 - 4⋅w₃) + z_1,1⋅log⎝1 - ℯ      ⎠ + z_1,2⋅log⎝1 - ℯ        ⎠ + z_1,3⋅log⎝1 -

                                                                 
  -2⋅w₁⋅w₃⎞            ⎛     -2⋅w₁⋅w₂⎞            ⎛     -2⋅w₁⋅w₃⎞
 ℯ        ⎠ + z_2,1⋅log⎝1 - ℯ        ⎠ + z_3,1⋅log⎝1 - ℯ        ⎠

# Algebra of Prior for Weights

In [49]:
bfry_density.subs(s,w(k))

       ⎛         \sigma__________    ⎞                         
       ⎜        -    ╲╱ K⋅\sigma     ⎟  -\sigma - 1     -τ⋅w(k)
\sigma⋅⎝- w(k)⋅ℯ                  + 1⎠⋅w           (k)⋅ℯ       
───────────────────────────────────────────────────────────────
 ⎛                                  \sigma⎞                    
 ⎜   \sigma   ⎛    \sigma__________⎞      ⎟                    
 ⎝- τ       + ⎝τ +     ╲╱ K⋅\sigma ⎠      ⎠⋅\Gamma(-\sigma + 1)

In [50]:
prior_of_weights = prior_of_weights.subs(lamb(theta(k)),1).subs(K,2)
log_prior_w1 = collectTermsWithVariable(expand_log(log(prior_of_weights.doit()),force=True),w(1))
expand_log_prior_w1 = expand_log(log_prior_w1.subs(bfry_density_function(w(1)),bfry_density.subs(s,w(1))),force=True)
expand_log_prior_w1 = collectTermsWithVariable(expand_log_prior_w1,w(1))
backToSubIndice(expand_log_prior_w1,w,(1,4,1))

                                   ⎛       \sigma__________    ⎞
                                   ⎜      -    ╲╱ K⋅\sigma     ⎟
-τ⋅w₁ + (-\sigma - 1)⋅log(w₁) + log⎝- w₁⋅ℯ                  + 1⎠

In [ ]:
integrate(gamma_distribution*log(1-exp(-x)),(x,0,oo))

In [57]:
integrate(gamma_distribution,(x,0,oo))

⎧     \alpha      -\alpha + 1                                                 
⎪\beta      ⋅\beta           ⋅Γ(\alpha)      ⎛                                
⎪──────────────────────────────────────  for ⎜-re(\alpha) + 1 < 1 ∧ │periodic_
⎪         \beta⋅\Gamma(\alpha)               ⎝                                
⎪                                                                             
⎪∞                                                                            
⎨⌠                                                                            
⎪⎮      \alpha  \alpha - 1  -\beta⋅x                                          
⎪⎮ \beta      ⋅x          ⋅ℯ                                                  
⎪⎮ ───────────────────────────────── dx                                       
⎪⎮           \Gamma(\alpha)                                                   
⎪⌡                                                                            
⎩0                                                  

In [58]:
gamma_distribution = ((beta**(alpha))/G(alpha))*(x**(alpha-1))*(exp(-beta*x))

In [59]:
gamma_distribution

     \alpha  \alpha - 1  -\beta⋅x
\beta      ⋅x          ⋅ℯ        
─────────────────────────────────
          \Gamma(\alpha)         

# Variational Lower Bound


We require for the variational approximation of the Graph and approximation for $q(\*W)$

\begin{equation}
L = \mathbb{E}_{q(\*W)}[P(\*Z,\*W)] - \mathbb{E}_{q(\*W)}[P(\*W)]
\end{equation}

\begin{equation}
L = \mathbb{E}_{q(\*W)}[P(\*Z|\*W)] - \text{KL}[q(\*W)|P(\*W)]
\end{equation}

# BFRY

$\text{BRS}(c,\tau,\sigma) = \frac{\sigma s^{- \sigma - 1} \left(- s e^{- \left(\frac{\sigma}{c}\right)^{\frac{1}{\sigma}}} + 1\right) e^{- \tau \theta}}{\left(- \tau^{\sigma} + \left(\tau + \left(\frac{\sigma}{c}\right)^{\frac{1}{\sigma}}\right)^{\sigma}\right) \Gamma{\left (- \sigma + 1 \right )}}$

In [11]:
#==========================
# GRAPH VARIABLES
#==========================
Da = Symbol("D^{*}_{\\alpha}")
Da

#==========================
# Densities variables
#==========================
Indicator = Symbol("\\mathbb{1}")
Indicator_tilted = Symbol("\\mathbb{1}_{\{ \\tau + (\\frac{\\alpha}{c})^{\\frac{1}{\\alpha}} \leq t \leq \\tau^{-1} \}}")

In [12]:
tau

τ

In [13]:
((alpha*(t**(-alpha-1))*Indicator_tilted)/((tau+(alpha/c)**(1/alpha))**alpha - tau**alpha))

                                                                              
\alpha⋅\mathbb{1}_{\{ \tau + (\frac{\alpha}{c})__{\frac{1}{\alpha}} \leq t \le
──────────────────────────────────────────────────────────────────────────────
                                                                    \alpha    
                                              ⎛            ________⎞          
                                     \alpha   ⎜           ╱ \alpha ⎟          
                                  - τ       + ⎜τ + \alpha╱  ────── ⎟          
                                              ⎝        ╲╱     c    ⎠          

                  -\alpha - 1
q \tau__{-1} \}}⋅t           
─────────────────────────────
                             
                             
                             
                             
                             

In [14]:
tilted_no_indicator = ((alpha*(t**(-alpha-1)))/((tau+(alpha/c)**(1/alpha))**alpha - tau**alpha))

In [15]:
tilted_no_indicator

                  -\alpha - 1           
          \alpha⋅t                      
────────────────────────────────────────
                                  \alpha
            ⎛            ________⎞      
   \alpha   ⎜           ╱ \alpha ⎟      
- τ       + ⎜τ + \alpha╱  ────── ⎟      
            ⎝        ╲╱     c    ⎠      

In [16]:
eq = Eq(tilted_no_indicator,y)

In [17]:
tilted_no_indicator_integrated = integrate(tilted_no_indicator,(t,x,b))
tilted_no_indicator_integrated

       ⎛⎧  log(b)    for -\alpha - 1 = -1⎞          ⎛⎧  log(x)    for -\alpha 
       ⎜⎪                                ⎟          ⎜⎪                        
       ⎜⎪  -\alpha                       ⎟          ⎜⎪  -\alpha               
\alpha⋅⎜⎨-b                              ⎟   \alpha⋅⎜⎨-x                      
       ⎜⎪──────────       otherwise      ⎟          ⎜⎪──────────       otherwi
       ⎜⎪  \alpha                        ⎟          ⎜⎪  \alpha                
       ⎝⎩                                ⎠          ⎝⎩                        
────────────────────────────────────────── - ─────────────────────────────────
                                   \alpha                                     
             ⎛            ________⎞                       ⎛            _______
    \alpha   ⎜           ╱ \alpha ⎟              \alpha   ⎜           ╱ \alpha
 - τ       + ⎜τ + \alpha╱  ────── ⎟           - τ       + ⎜τ + \alpha╱  ──────
             ⎝        ╲╱     c    ⎠                 

In [18]:
tilted_no_indicator_integrated.args[0].args[-1].args[1][0]

  -\alpha 
-b        
──────────
  \alpha  

In [19]:
solve(eq,t)

⎡                                                 -1     ⎤
⎢                                              ──────────⎥
⎢                                              \alpha + 1⎥
⎢⎛  ⎛                                  \alpha⎞⎞          ⎥
⎢⎜  ⎜            ⎛            ________⎞      ⎟⎟          ⎥
⎢⎜  ⎜   \alpha   ⎜           ╱ \alpha ⎟      ⎟⎟          ⎥
⎢⎜y⋅⎜- τ       + ⎜τ + \alpha╱  ────── ⎟      ⎟⎟          ⎥
⎢⎜  ⎝            ⎝        ╲╱     c    ⎠      ⎠⎟          ⎥
⎢⎜────────────────────────────────────────────⎟          ⎥
⎣⎝                   \alpha                   ⎠          ⎦

# BFRY Densities

In [20]:
Numerator = (alpha*s**(-alpha-1)*(exp(-tau*s))*(1-exp(-(alpha/c)**(1/alpha))*s))
Denominator = G(1-alpha)*((tau+(alpha/c)**(1/alpha))**alpha - tau**alpha)
bfry_density = (Numerator/Denominator)

In [22]:
bfry_density.subs(alpha,sig).subs(s,w)

                        ⎛              ________    ⎞          
                        ⎜             ╱ \sigma     ⎟          
                        ⎜     -\sigma╱  ──────     ⎟          
            -\sigma - 1 ⎜          ╲╱     c        ⎟  -τ⋅w    
    \sigma⋅w           ⋅⎝- w⋅ℯ                  + 1⎠⋅ℯ        
──────────────────────────────────────────────────────────────
⎛                                  \sigma⎞                    
⎜            ⎛            ________⎞      ⎟                    
⎜   \sigma   ⎜           ╱ \sigma ⎟      ⎟                    
⎜- τ       + ⎜τ + \sigma╱  ────── ⎟      ⎟⋅\Gamma(-\sigma + 1)
⎝            ⎝        ╲╱     c    ⎠      ⎠                    

In [23]:
bfry_density_function = Symbol("g_{1/K,\sigma}",function=True)
bfry_density_function

g_{1/K,\sigma}

# Dirichlet

In [24]:
dirichlet = (G(n+1)/Product(G(n(k)+1),(k,1,K)))*Product(pk(k)**n(k),(k,1,K))

In [25]:
dirichlet.subs(K,3).subs(p(k),w(k))

                3            
              ┬────┬         
              │    │  n(k)   
\Gamma(n + 1)⋅│    │ w    (k)
              │    │         
              k = 1          
─────────────────────────────
      3                      
    ┬───┬                    
    │   │ \Gamma(n(k) + 1)   
    │   │                    
    k = 1                    

# Joint Model Likehood

In [26]:
probability_of_fnrm = Product(bfry_density_function(w(k))*lamb(theta(k))*exp(-u*w(k)),(k,1,K))
probability_of_fnrm = u**(Da-1)*probability_of_fnrm
probability_of_fnrm

                       K                                                    
                     ┬────┬                                                 
 D_{\alpha}__{*} - 1 │    │                                          -u⋅w(k)
u                   ⋅│    │ \lambda(\theta(k))⋅g_{1/K,\sigma}(w(k))⋅ℯ       
                     │    │                                                 
                     k = 1                                                  

In [27]:
probability_of_fnrm

                       K                                                    
                     ┬────┬                                                 
 D_{\alpha}__{*} - 1 │    │                                          -u⋅w(k)
u                   ⋅│    │ \lambda(\theta(k))⋅g_{1/K,\sigma}(w(k))⋅ℯ       
                     │    │                                                 
                     k = 1                                                  

In [28]:
join_model = probability_of_fnrm*dirichlet.subs(p(k),w(k))
join_model

                                   ⎛  K                                       
                                   ⎜┬────┬                                    
 D_{\alpha}__{*} - 1               ⎜│    │                                    
u                   ⋅\Gamma(n + 1)⋅⎜│    │ \lambda(\theta(k))⋅g_{1/K,\sigma}(w
                                   ⎜│    │                                    
                                   ⎝k = 1                                     
──────────────────────────────────────────────────────────────────────────────
                                             K                                
                                           ┬───┬                              
                                           │   │ \Gamma(n(k) + 1)             
                                           │   │                              
                                           k = 1                              

             ⎞   K            
             ⎟ ┬────

In [29]:
expand_log(log(dirichlet.subs(K,3).doit()),force=True)

n(1)⋅log(p(1)) + n(2)⋅log(p(2)) + n(3)⋅log(p(3)) + log(\Gamma(n + 1)) - log(\G
amma(n(1) + 1)) - log(\Gamma(n(2) + 1)) - log(\Gamma(n(3) + 1))

In [32]:
log_dirichlet = expand_log(log(dirichlet.subs(K,3).doit()),force=True)
backToSubIndice(backToSubIndice(backToSubIndice(log_dirichlet,p,(1,4)),x,(1,4)),n,(1,4))

n₁⋅log(p₁) + n₂⋅log(p₂) + n₃⋅log(p₃) + log(\Gamma(n + 1)) - log(\Gamma(n₁ + 1)
) - log(\Gamma(n₂ + 1)) - log(\Gamma(n₃ + 1))

In [31]:
def collectTermsWithVariable(expression,variable):
    return Add(*[argi for argi in expression.args if argi.has(variable)])

def subsParentesis(expresion,p,w,krange):
    for k in range(*krange):
        expresion = expresion.subs(p(k),w(k))
    return expresion

def subsParentesisDouble(expresion,p,w,krange):
    for u in range(*krange):
        for v in range(*krange):
            expresion = expresion.subs(p(u,v),w(u,v))
    return expresion

def backToSubIndice(expresion,p,krange):
    for k in range(*krange):
        expresion = expresion.subs(p(k),Symbol(str(p)+"_{0}".format(k)))
    return expresion

def backToSubIndiceDouble(expresion,p,krange):
    for u in range(*krange):
        for v in range(*krange):
            expresion = expresion.subs(p(u,v),Symbol(str(p)+"_{0},{1}".format(u,v)))
    return expresion

def backToSuperIndice(expresion,p,krange):
    for k in range(*krange):
        expresion = expresion.subs(p(k),Symbol(str(p)+"^{0}".format(k)))
    return expresion